In [1]:
% run 1-datasource.ipynb

In [2]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import torch.nn.functional as F
from skorch.net import NeuralNetClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [4]:
for _, i in icebergs.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [5]:
y = icebergs.is_iceberg.values
x = (
    np.stack(
        [
            scaler_1.transform(np.stack(icebergs.band_1)),
            scaler_2.transform(np.stack(icebergs.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).astype(np.float32)

In [6]:
x.dtype, y.dtype

(dtype('float32'), dtype('int64'))

In [7]:
type(x), type(y)

(numpy.ndarray, numpy.ndarray)

In [8]:
y.shape, x.shape

((1604,), (1604, 2, 75, 75))

In [9]:
insize1 = 2 
outsize1 = 32 
outsize2 = 32

class Net(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super().__init__()
        

        self.layer1 = nn.Sequential(
            nn.Conv2d(insize1, outsize1, kernel_size=7, stride=1, padding=2, groups=2),
            nn.BatchNorm2d(outsize1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(outsize1, outsize2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(2592, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Softmax(1)
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
net = NeuralNetClassifier(
    Net,
    optimizer=torch.optim.Adam,
    criterion=nn.CrossEntropyLoss,
    max_epochs=10,
    batch_size=25,
    lr=0.000001,
    use_cuda=True
)

In [11]:
net.fit(x, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6931       0.5994        0.6889  2.2235
      2        0.6877       0.5776        0.6838  1.1326
      3        0.6838       0.5745        0.6794  1.1357
      4        0.6805       0.5776        0.6756  1.2032
      5        0.6775       0.5870        0.6721  1.1298
      6        0.6748       0.5901        0.6690  1.1283
      7        0.6723       0.5901        0.6660  1.1263
      8        0.6699       0.6056        0.6633  1.1498
      9        0.6677       0.6025        0.6606  1.1278
     10        0.6656       0.6056        0.6581  1.1321


<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=Net(
    (layer1): Sequential(
      (0): Conv2d (2, 32, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2), groups=2)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(4, 4), stride=(4, 4), dilation=(1, 1))
    )
    (layer2): Sequential(
      (0): Conv2d (32, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    )
    (fc): Sequential(
      (0): Linear(in_features=2592, out_features=1024)
      (1): ReLU()
      (2): Linear(in_features=1024, out_features=512)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=2)
      (5): Softmax()
    )
  ),
)

In [ ]:
%%capture output
from sklearn.model_selection import GridSearchCV


params = {
    'lr': [0.000001, 0.000002,0.000004,0.000008],
    'max_epochs': [50,100,150, 200,250],
    'module__num_units': [5,10,15,20,40,80],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='neg_log_loss',n_jobs=1)

In [ ]:
gs.fit(x, y)

Re-initializing module!
Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6966       0.4977        0.6931  0.7739
      2        0.6891       0.6233        0.6855  0.7584
      3        0.6841       0.5860        0.6802  0.7609
      4        0.6804       0.5581        0.6760  0.7506
      5        0.6774       0.5628        0.6726  0.7486
      6        0.6750       0.5581        0.6698  0.7475
      7        0.6729       0.5581        0.6672  0.7604
      8        0.6710       0.5581        0.6650  0.7591
      9        0.6693       0.5581        0.6629  0.7482
     10        0.6678       0.5581        0.6609  0.7480
     11        0.6663       0.5581        0.6591  0.7536
     12        0.6649       0.5581        0.6574  0.7665
     13        0.6636       0.5581        0.6558  0.7594
     14        0.6623       0.5581        0.6543  0.7525
     15        0.6610       0.5581      

In [ ]:
%%capture output2
print(gs.best_score_, gs.best_params_)

In [ ]:
class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=F.relu):
        super(MyModule, self).__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

In [ ]:
class BCL2(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.loss = nn.BCEWithLogitsLoss(*args, **kwargs)
        
    def forward(self, input, target, *args, **kwargs):
        return self.loss.forward(input, target.view(-1, 1), *args, **kwargs)
    
    def backward(self, grad_output, *args, **kwargs):
        return self.loss.backward(grad_output, *args, **kwargs)

In [17]:
output2.show()

-0.355908236186 {'max_epochs': 100, 'lr': 4e-06, 'module__num_units': 80}


In [20]:
output.show()

In [43]:
import scipy.stats as sc
param_dis = {
    'lr': sc.uniform(loc=0.000001,scale=0.00001),
    'max_epochs': sc.randint(low=50,high=250)
}

In [47]:
from sklearn.model_selection import RandomizedSearchCV
gsR = RandomizedSearchCV(net, param_dis, refit=False, scoring='neg_log_loss',n_jobs=1,n_iter=5)

In [48]:
gsR.fit(x,y)

Re-initializing module!
Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6882       0.5674        0.6838  1.0834
      2        0.6777       0.5581        0.6705  0.7819
      3        0.6727       0.5581        0.6640  0.7701
      4        0.6695       0.5581        0.6594  0.7523
      5        0.6669       0.5581        0.6556  0.7527
      6        0.6646       0.5581        0.6523  0.7509
      7        0.6625       0.5581        0.6494  0.7562
      8        0.6604       0.5581        0.6467  0.7563
      9        0.6584       0.5721        0.6442  0.7594
     10        0.6565       0.5721        0.6419  0.7581
     11        0.6546       0.5767        0.6396  0.7489
     12        0.6527       0.5814        0.6373  0.7524
     13        0.6507       0.5860        0.6351  0.7608
     14        0.6488       0.6000        0.6329  0.7529
     15        0.6468       0.6233      

     22        0.6335       0.6028        0.6396  0.7485
     23        0.6315       0.6028        0.6380  0.7719
     24        0.6295       0.6168        0.6364  0.7510
     25        0.6274       0.6215        0.6347  0.7463
     26        0.6254       0.6308        0.6331  0.7447
     27        0.6234       0.6308        0.6315  0.7454
     28        0.6213       0.6308        0.6298  0.7495
     29        0.6193       0.6355        0.6281  0.7503
     30        0.6172       0.6449        0.6264  0.7507
     31        0.6151       0.6495        0.6247  0.7463
     32        0.6130       0.6542        0.6229  0.7518
     33        0.6108       0.6589        0.6212  0.7487
     34        0.6087       0.6636        0.6194  0.7468
     35        0.6065       0.6589        0.6176  0.7449
     36        0.6043       0.6589        0.6159  0.7495
     37        0.6021       0.6542        0.6141  0.7560
     38        0.5998       0.6542        0.6122  0.7489
     39        0.5975       0.6

     45        0.4760       0.7023        0.5559  0.7486
     46        0.4713       0.7023        0.5524  0.7502
     47        0.4666       0.7116        0.5490  0.7536
     48        0.4619       0.7209        0.5456  0.7589
     49        0.4571       0.7256        0.5421  0.7539
     50        0.4523       0.7256        0.5386  0.7505
     51        0.4475       0.7302        0.5351  0.7531
     52        0.4427       0.7349        0.5316  0.7538
     53        0.4379       0.7349        0.5282  0.7487
     54        0.4331       0.7442        0.5247  0.7471
     55        0.4282       0.7442        0.5212  0.7532
     56        0.4234       0.7488        0.5178  0.7520
     57        0.4186       0.7488        0.5145  0.7522
     58        0.4138       0.7535        0.5111  0.7522
     59        0.4090       0.7535        0.5078  0.7603
     60        0.4042       0.7535        0.5045  0.7605
     61        0.3994       0.7581        0.5012  0.7544
     62        0.3946       0.7

     67        0.4163       0.8326        0.4426  0.7548
     68        0.4112       0.8279        0.4396  0.7621
     69        0.4062       0.8279        0.4367  0.7654
     70        0.4012       0.8326        0.4338  0.7582
     71        0.3962       0.8372        0.4309  0.7578
     72        0.3913       0.8419        0.4281  0.7488
     73        0.3864       0.8372        0.4254  0.7516
     74        0.3816       0.8372        0.4227  0.7493
     75        0.3768       0.8419        0.4201  0.7597
     76        0.3720       0.8419        0.4176  0.7519
     77        0.3673       0.8372        0.4151  0.7534
     78        0.3627       0.8326        0.4127  0.7673
     79        0.3580       0.8326        0.4102  0.7617
     80        0.3534       0.8326        0.4078  0.7534
     81        0.3488       0.8326        0.4054  0.7525
     82        0.3444       0.8326        0.4031  0.7542
     83        0.3399       0.8372        0.4009  0.7531
     84        0.3355       0.8

     56        0.4818       0.7523        0.5138  0.7610
     57        0.4776       0.7523        0.5107  0.7565
     58        0.4733       0.7570        0.5077  0.7493
     59        0.4689       0.7570        0.5046  0.7492
     60        0.4646       0.7664        0.5016  0.7506
     61        0.4602       0.7710        0.4986  0.7594
     62        0.4558       0.7757        0.4956  0.7706
     63        0.4514       0.7757        0.4926  0.7583
     64        0.4469       0.7757        0.4895  0.7510
     65        0.4425       0.7710        0.4865  0.7551
     66        0.4380       0.7850        0.4835  0.7514
     67        0.4334       0.7850        0.4805  0.7578
     68        0.4288       0.7850        0.4774  0.7559
     69        0.4241       0.7850        0.4744  0.7577
     70        0.4195       0.7897        0.4714  0.7526
     71        0.4149       0.7944        0.4685  0.7623
     72        0.4103       0.7944        0.4655  0.7597
     73        0.4057       0.7

...

    157        0.0124       0.8512        0.3736  0.7513
    158        0.0120       0.8512        0.3750  0.7528
    159        0.0117       0.8512        0.3763  0.7477
    160        0.0113       0.8512        0.3775  0.7450
    161        0.0109       0.8512        0.3788  0.7700
    162        0.0106       0.8512        0.3802  0.7497
    163        0.0103       0.8512        0.3814  0.7555
    164        0.0099       0.8512        0.3828  0.7502
    165        0.0096       0.8512        0.3840  0.7549
    166        0.0093       0.8512        0.3854  0.7490
    167        0.0090       0.8512        0.3867  0.7592
    168        0.0088       0.8512        0.3880  0.7529
    169        0.0085       0.8512        0.3894  0.7548
    170        0.0082       0.8512        0.3907  0.7504
    171        0.0080       0.8512        0.3920  0.7490
    172        0.0077       0.8512        0.3933  0.7452
    173        0.0075       0.8512        0.3946  0.7477
    174        0.0073       0.8

     53        0.3027       0.8364        0.3929  0.7557
     54        0.2953       0.8364        0.3891  0.7565
     55        0.2878       0.8411        0.3856  0.7542
     56        0.2806       0.8364        0.3821  0.7556
     57        0.2734       0.8364        0.3789  0.7581
     58        0.2665       0.8318        0.3756  0.7587
     59        0.2596       0.8318        0.3726  0.7537
     60        0.2529       0.8318        0.3696  0.7571
     61        0.2464       0.8318        0.3668  0.7599
     62        0.2400       0.8364        0.3641  0.7557
     63        0.2338       0.8364        0.3615  0.7599
     64        0.2277       0.8364        0.3591  0.7563
     65        0.2218       0.8411        0.3567  0.7578
     66        0.2159       0.8458        0.3544  0.7558
     67        0.2102       0.8458        0.3522  0.7589
     68        0.2046       0.8458        0.3501  0.7822
     69        0.1991       0.8458        0.3481  0.7551
     70        0.1938       0.8

    170        0.0070       0.8411        0.3627  0.7545
    171        0.0068       0.8411        0.3635  0.7608
    172        0.0066       0.8458        0.3644  0.7606
    173        0.0064       0.8411        0.3653  0.7567
    174        0.0062       0.8411        0.3661  0.7540
    175        0.0060       0.8411        0.3670  0.7591
    176        0.0058       0.8411        0.3678  0.7585
    177        0.0056       0.8411        0.3687  0.7572
    178        0.0055       0.8411        0.3695  0.7544
    179        0.0053       0.8411        0.3704  0.7594
    180        0.0052       0.8411        0.3712  0.7590
    181        0.0050       0.8411        0.3721  0.7533
    182        0.0049       0.8411        0.3730  0.7591
    183        0.0047       0.8411        0.3739  0.7573
    184        0.0046       0.8411        0.3747  0.7650
    185        0.0045       0.8411        0.3756  0.7621
    186        0.0043       0.8411        0.3765  0.7584
    187        0.0042       0.8

     64        0.1028       0.8140        0.3895  0.7590
     65        0.0988       0.8140        0.3890  0.7563
     66        0.0949       0.8140        0.3886  0.7543
     67        0.0911       0.8140        0.3883  0.7538
     68        0.0875       0.8047        0.3881  0.7536
     69        0.0840       0.8140        0.3879  0.7534
     70        0.0807       0.8140        0.3878  0.7513
     71        0.0775       0.8140        0.3879  0.7521
     72        0.0745       0.8140        0.3878  0.7589
     73        0.0716       0.8140        0.3878  0.7610
     74        0.0688       0.8186        0.3880  0.7537
     75        0.0661       0.8186        0.3883  0.7539
     76        0.0636       0.8186        0.3886  0.7871
     77        0.0611       0.8186        0.3891  0.7580
     78        0.0588       0.8186        0.3896  0.7575
     79        0.0565       0.8186        0.3903  0.7505
     80        0.0543       0.8233        0.3909  0.7516
     81        0.0523       0.8

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=Net(
    (layer1): Sequential(
      (0): Conv2d (2, 32, kernel_size=(7, 7), stride=(1, 1), padding=(2, 2), groups=2)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
      (2): ReLU()
      (3): MaxPool2d(ker...    (3): ReLU()
      (4): Linear(in_features=512, out_features=2)
      (5): Softmax()
    )
  ),
),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'max_epochs': array([216, 186,  80, 180, 116]), 'lr': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fabacef4f28>, 'module__num_units': [5, 10, 15, 20, 40, 80]},
          pre_dispatch='2*n_jobs', random_state=None, refit=False,
          return_train_score='warn', scoring='neg_log_loss', verbose=0)

In [ ]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X), dim=-1)
        return X

In [51]:
print(gs.best_score_, gs.best_params_)

-0.355908236186 {'max_epochs': 100, 'lr': 4e-06, 'module__num_units': 80}
